<a href="https://colab.research.google.com/github/SytherAsh/CyberScan_CyberSecurity_ChatBot/blob/main/RAG_Retrieval_FAISS_Groq_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community faiss-cpu transformers pypdf langchain_groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting

In [2]:
# # prompt: mount drivw

# from google.colab import drive
# drive.mount('/content/drive')

API
--

In [2]:
from google.colab import userdata
groq=userdata.get("Groq")
gemma=userdata.get("Gemma")
huggingFace=userdata.get("HuggingFace")

Library
--

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

PDF_Reader
--

In [4]:
loader=PyPDFLoader("/content/Cybersecurity Report Template.pdf")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)

In [6]:
# len(final_documents)
# doc=final_documents[1]
# print(doc.page_content)



HuggingFace Embedding
--

In [5]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    # model_name="BAAI/bge-small-en-v1.5",
    model_name="sentence-transformers/all-MiniLM-l6-v2",

    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

VectorDB FAISS
--

In [6]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents,huggingface_embeddings)
vectorstore.save_local("/content/drive/MyDrive/Colab Notebooks/CyberScan/faiss_index")

In [7]:
# # Query using Similarity Search


# query="what are major community partners  "
# relevant_docments=vectorstore.similarity_search(query)

# print(relevant_docments[0])

Retriever
--

In [8]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ee5793abf10> search_kwargs={'k': 3}


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']=huggingFace

HuggingFace LLM
--

In [17]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.5,"max_length":1000}

)
query="How are we securing our network perimeter, including firewalls, intrusion detection/prevention systems (IDS/IPS), and VPNs?"
text=hf.invoke(query)

def format_text(text):
    # Remove any leading/trailing whitespace
    formatted_text = text.strip()

    # Replace multiple spaces with a single space
    formatted_text = " ".join(formatted_text.split())

    # Convert text to title case (capitalize the first letter of each word)
    formatted_text = formatted_text.title()

    # Add line breaks after periods for readability
    formatted_text = formatted_text.replace(". ", ".\n")

    return formatted_text

# Example usage
formatted_text = format_text(text)
print(formatted_text)


How Are We Securing Our Network Perimeter, Including Firewalls, Intrusion Detection/Prevention Systems (Ids/Ips), And Vpns? We Have A Very Good Firewall.
We Have A Very Good Intrusion Detection/Prevention System.
We Have A Very Good Vpn.
We Have A Very Good Anti-Virus System.
We Have A Very Good Anti-Spyware System.
We Have A Very Good Anti-Malware System.
We Have A Very Good Anti-Botnet System.
We Have A Very Good Anti-Phishing System.
We Have A Very Good Anti-Spam System.
We Have


RetrievalQ&A
--

In [27]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template="""
Use the following piece of context to answer the question asked.
give me a summary base on the documents uploaded
Give me only the answer

context:{context}
Question:{question}
Answer:
"""


In [28]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])


In [29]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [34]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
res = result['result']

# Try to split the result if it contains specific markers like 'Answer:'
# Adjust the split text based on your actual format.
def arrange(res):
  if "Answer:" in res:
      answer = res.split("Answer:")[1].strip()
  elif "answer:" in res:
      answer = res.split("answer:")[1].strip()
  else:
      # Fallback to the entire result if no split pattern is found
      answer = res
  return answer

# answer=arrange(res)
# print(answer)

# print(result["result"])

In [31]:
cybersecurity_questions_dict = {
    "Network Security": "How are we securing our network perimeter, including firewalls, intrusion detection/prevention systems (IDS/IPS), and VPNs?",
    "Identity and Access Management (IAM)": "Do we have robust multi-factor authentication (MFA) and least-privilege access policies for all users and critical systems?",
    "Incident Response": "Do we have a tested incident response plan, and are all employees aware of their roles during a security breach?",
    "Cloud Security": "Are we adequately securing our cloud environments, including data encryption, access control, and continuous monitoring for threats?",
    "Endpoint Security": "Are all endpoints (including remote devices and employee mobile devices) protected with endpoint detection and response (EDR) tools, antivirus, and encryption?",
    "Data Security": "Are we encrypting sensitive data both at rest and in transit, and do we have a data loss prevention (DLP) strategy in place?",
    "Application Security": "Are our applications thoroughly tested for vulnerabilities (e.g., through penetration testing and regular security updates) before deployment?",
    "Compliance and Governance": "Are we meeting all applicable regulatory requirements and industry standards for cybersecurity (e.g., GDPR, HIPAA, PCI-DSS)?",
    "Risk Management": "Have we conducted a comprehensive risk assessment to identify and prioritize our most critical cyber risks, and do we have a mitigation strategy in place?",
    "Threat Intelligence": "Are we actively leveraging threat intelligence to stay informed about emerging threats and vulnerabilities, and how are we integrating this intelligence into our defense strategies?"
}


In [35]:
# Function to query your RAG bot and get responses
def get_rag_responses(retrievalQA, questions_dict):
    responses = {}
    for domain, question in questions_dict.items():
        query = question
        result = retrievalQA.invoke({"query": query})
        res = result['result']
        res=arrange(res)
        responses[domain] = res


    return responses

# Assuming `retrievalQA` is the instance of your RAG bot, you would call the function like this:
responses = get_rag_responses(retrievalQA, cybersecurity_questions_dict)

# Printing the responses for each domain
for domain, answer in responses.items():
    print(f"Domain: {domain}")
    print(f"Question: {cybersecurity_questions_dict[domain]}")
    print(f"Answer: {answer}\n")

Domain: Network Security
Question: How are we securing our network perimeter, including firewalls, intrusion detection/prevention systems (IDS/IPS), and VPNs?
Answer: Our network perimeter is secured using a combination of ﬁrewalls, intrusion detection/prevention
systems (IDS/IPS), and VPNs. We have implemented a robust ﬁrewall solution to control inbound
and outbound network traffic, and we regularly update the ﬁrewall rules to ensure they are
effective against the latest threats. Additionally, we have deployed IDS/IPS solutions to monitor
network activity and detect any suspicious or malicious behavior.

Domain: Identity and Access Management (IAM)
Question: Do we have robust multi-factor authentication (MFA) and least-privilege access policies for all users and critical systems?
Answer: We have implemented robust MFA and least-privilege access policies for all users and critical systems. All users are required to use MFA when accessing corporate resources, including email, VPN, and 

In [18]:

# import joblib

# # Save the model
# joblib.dump(retrievalQA, "retrieverQA_model.pkl")


SymtemPrompt & LLM
--

In [19]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq


llm=ChatGroq(groq_api_key=groq,model_name='Gemma-7b-it')

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "

    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)

answer=chain.invoke({"input": query})

In [20]:
print(answer['answer'])

The provided text does not contain any information regarding the measures ABC Corp takes to secure its network perimeter, including firewalls, intrusion detection/prevention systems (IDS/IPS), and VPNs, so I am unable to answer this question from the given context.


In [22]:
# import joblib

# # Save the model
# joblib.dump(create_retrieval_chain, "chain_model.pkl")
